<a href="https://colab.research.google.com/github/benson4250/Badminton/blob/main/%E3%80%90Demo04%E3%80%91%E7%94%A8AISuite%E6%89%93%E9%80%A0%E5%93%A1%E7%91%9B%E5%BC%8F%E6%80%9D%E8%80%83%E7%94%9F%E6%88%90%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 申請自己的 API 金鑰

不管用哪一個供應商的服務, 基本上都需要他們的 API 鑰, 可向下面幾家申請。

#### (1) Mistral AI 金鑰 (可免費使用)

請至 https://console.mistral.ai/ 註冊，並選擇 plan (可以選免費的), 接著就可以申請 Mistral AI 的金鑰。請把這個鑰存在左方鑰匙的部份, 以 "Mistral" 的名稱存起來。



#### (2) OpenAI API 金鑰

OpenAI 現在沒有免費的 quota 可以使用, 所以要用 OpenAI 的模型, 請自行儲值。一般練習 5 美金就很足夠。

[`https://platform.openai.com`](https://platform.openai.com)

請把這個鑰存在左方鑰匙的部份, 以 "OpenAI" 的名稱存起來。

#### (3) 使用 Groq 金鑰 (可免費使用)

Groq 最大的特點是速度很快, 而且可以免費使用 (只是有流量限制), 企業可以付費使用, 能用許多開源型的 LLM。請至 https://console.groq.com/ 註冊並申請金鑰。


#### 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 30.8 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.3
    Uninstalling httpx-sse-0.4.3:
      Successfully uninstalled httpx-sse-0.4.3
  Attempting uninstall: docstring-parser
    Found existing installation: docstring_parser 0.17.0
    Uninstalling docstring_parser-0.17.0:
      Successfully uninstalled docstring_parser-0.17.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstall

In [2]:
import aisuite as ai

In [3]:
import os
from google.colab import userdata

In [5]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama-3.3-70b-versatile"

### 2. 打造員瑛式思考生成器

ChatGPT API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

基本上過去的對話紀錄長這個樣子。

    messages = [{"role":"system", "content":"ChatGPT的「人設」"},
            {"role": "user", "content": "使用者說"},
            {"role": "assistant", "content": "ChatGPT回應"},
            ：
            ：
            {"role": "user", "content": prompt (最後說的)}]

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hi",
          provider="groq",
          model="llama-3.3-70b-versatile"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

請先為你的對話機器人做角色設定。

In [7]:
system="""
請用台灣習慣的中文來寫這段 po 文：
請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情,
用我的第一人稱、社群媒體 po 文的口吻說一次,
說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。
可以適度的加上 emoji。
"""

試用一下 (預設的 Llama 3 70B)

In [8]:
prompt = "今天咖啡灑到電腦上了!"
print(reply(system=system, prompt=prompt))

哇哈哈 😊，今天發生了一件超級特別的事！ 🤩 我正在享用美味的咖啡，結果咖啡竟然灑到了我的電腦上 🤦‍♀️！雖然听起來好像是件坏事，但我覺得這其實是一件超幸運的事 🍀！因為，如果咖啡灑到其他東西上，可能會造成更大的損壞，但我的電腦竟然還是可以使用 😌。而且，這也給了我一個機會，可以把我的電腦好好清潔一下 🧹，讓它變得更乾淨、更安全 🙏。這一切都證明，即使看似不幸的事，也可能帶來意想不到的好處 🌟。完全是 Lucky Vicky 呀! 💫


選另一個模型試試。

In [19]:
# provider="openai"
# model="gpt-4o"

provider="groq"
model="llama-3.3-70b-versatile"

In [20]:
print(reply(system=system, prompt=prompt, provider=provider, model=model))

今天超開心的事就是我的電腦遇到了一點小意外 🤦‍♀️，結果咖啡灑到了電腦上啦！😂 這其實是件超幸運的事，因為這讓我有機會好好整理電腦，清理一下這麼久沒動過的資料 📁，也讓我可以體驗一下電腦維修的過程 🛠️。這樣我就可以確保我的電腦以後會運作得更順暢、更高效 🚀。完全是 Lucky Vicky 呀! 😊


### 3. 用 Gradio 打造 Web App

我們先來安裝 `openai` 套件, 還有快速打造 Web App 的 `gradio`。

In [12]:
!pip install gradio

In [13]:
import gradio as gr

In [14]:
system="""
請用台灣習慣的中文來寫這段 po 文：
請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情,
用我的第一人稱、社群媒體 po 文的口吻說一次,
說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。
可以適度的加上 emoji。
"""

設定你要的模型。

In [15]:
provider="groq"
model="llama-3.3-70b-versatile"

# provider = "openai"
# model = "gpt-4o"

# provider = "groq"
# model = "gemma2-9b-it"

In [16]:
def lucky_post(prompt):
    response = reply(system=system,
                     prompt=prompt,
                     provider = provider,
                     model = model
                    )
    return response

In [17]:
with gr.Blocks(title="員瑛式思考產生器") as demo:
    gr.Markdown("### ꒰*ˊᵕˋ꒱ 員瑛式思考產生器 Lucky Vicky 🌈")
    gr.Markdown("請輸入一件你覺得超小事，甚至有點倒楣的事，讓我幫你用員瑛式思考，超正向的方式重新詮釋！")

    with gr.Row():
        user_input = gr.Textbox(label="今天發生的事情是…", placeholder="例如：今天出門就下大雨, 可是忘了帶傘...")

    submit_btn = gr.Button("Lucky Vicky 魔法!")
    output = gr.Textbox(label="📣 員瑛式貼文")

    submit_btn.click(fn=lucky_post, inputs=user_input, outputs=output)

In [18]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d2ddb8aeeb03057b77.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d2ddb8aeeb03057b77.gradio.live
